# Sales & Revenue Analysis with Python

This notebook analyzes a synthetic sales dataset using Python and pandas.
The goal is to demonstrate data loading, cleaning, transformation, and business-oriented analysis.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt